In [4]:
import os
import numpy as np
import sys
sys.path.insert(1, '/workspaces/baseline')

from railseg.pcd_processing import pcd_to_las

In [3]:
pcd_path = "/workspaces/baseline/data/semantic_kitti/dataset/sequences/00/velodyne/000000.bin"
with open(pcd_path, "rb") as b:
    scan = np.fromfile(b, dtype=np.float32).reshape(-1, 4)

In [4]:
scan

array([[ 5.2897942e+01,  2.2989739e-02,  1.9979945e+00,  7.9999998e-02],
       [ 5.3750526e+01,  1.9291429e-01,  2.0269539e+00,  0.0000000e+00],
       [ 5.3803116e+01,  3.6183926e-01,  2.0289137e+00,  0.0000000e+00],
       ...,
       [ 3.8401384e+00, -1.4381756e+00, -1.7735560e+00,  3.4000000e-01],
       [ 3.8257158e+00, -1.4192016e+00, -1.7645701e+00,  2.5000000e-01],
       [ 4.0923753e+00, -1.5071962e+00, -1.8955611e+00,  0.0000000e+00]],
      dtype=float32)

In [5]:
label_file = '/workspaces/baseline/data/semantic_kitti/dataset/sequences/00/labels/000000.label'
if os.path.exists(label_file):
    with open(label_file, "rb") as a:
        segment = np.fromfile(a, dtype=np.int32).reshape(-1)
        # segment = np.vectorize(self.learning_map.__getitem__)(
        #     segment & 0xFFFF
        # ).astype(np.int32)

In [6]:
(segment & 0xFFFF).astype(np.int32)

array([ 0,  0,  0, ..., 40, 40, 40], dtype=int32)

In [7]:
np.unique((segment & 0xFFFF).astype(np.int32))

array([  0,   1,  10,  40,  44,  48,  50,  51,  52,  60,  70,  71,  72,
        80,  81,  99, 255], dtype=int32)

Notice that it corresponds exactly with the label in pointcept/datasets/semantic_kitti.py in the mapping. I think we need to the the same with the point cloud from OSDaR. Set all the labels that are unknown to -1. Also not sure that the class have a number in osdar, so maybe attribute one.

In [8]:
len(scan)

124668

In [9]:
len(segment)

124668

In [5]:
ignore_index=20
learning_map = {
            0: ignore_index,  # "unlabeled"
            1: ignore_index,  # "outlier" mapped to "unlabeled" --------------------------mapped
            10: 0,  # "car"
            11: 1,  # "bicycle"
            13: 4,  # "bus" mapped to "other-vehicle" --------------------------mapped
            15: 2,  # "motorcycle"
            16: 4,  # "on-rails" mapped to "other-vehicle" ---------------------mapped
            18: 3,  # "truck"
            20: 4,  # "other-vehicle"
            30: 5,  # "person"
            31: 6,  # "bicyclist"
            32: 7,  # "motorcyclist"
            40: 8,  # "road"
            44: 9,  # "parking"
            48: 10,  # "sidewalk"
            49: 11,  # "other-ground"
            50: 12,  # "building"
            51: 13,  # "fence"
            52: ignore_index,  # "other-structure" mapped to "unlabeled" ------------------mapped
            60: 8,  # "lane-marking" to "road" ---------------------------------mapped
            70: 14,  # "vegetation"
            71: 15,  # "trunk"
            72: 16,  # "terrain"
            80: 17,  # "pole"
            81: 18,  # "traffic-sign"
            99: ignore_index,  # "other-object" to "unlabeled" ----------------------------mapped
            252: 0,  # "moving-car" to "car" ------------------------------------mapped
            253: 6,  # "moving-bicyclist" to "bicyclist" ------------------------mapped
            254: 5,  # "moving-person" to "person" ------------------------------mapped
            255: 7,  # "moving-motorcyclist" to "motorcyclist" ------------------mapped
            256: 4,  # "moving-on-rails" mapped to "other-vehicle" --------------mapped
            257: 4,  # "moving-bus" mapped to "other-vehicle" -------------------mapped
            258: 3,  # "moving-truck" to "truck" --------------------------------mapped
            259: 4,  # "moving-other"-vehicle to "other-vehicle" ----------------mapped
        }

In [9]:

data_path = "/workspaces/baseline/data/semantic_kitti/dataset/sequences/00/velodyne/000000.bin"
frame_nb=data_path.split("/")[-1].replace(".bin","")
with open(data_path, "rb") as b:
    scan = np.fromfile(b, dtype=np.float32).reshape(-1, 4)
coord = scan[:, :3]
strength = scan[:, -1].reshape([-1, 1])

label_file = data_path.replace("velodyne", "labels").replace(".bin", ".label")
if os.path.exists(label_file):
    with open(label_file, "rb") as a:
        segment = np.fromfile(a, dtype=np.int32).reshape(-1)
        segment = np.vectorize(learning_map.__getitem__)(
            segment & 0xFFFF
        ).astype(np.int32)
else:
    segment = np.zeros(scan.shape[0]).astype(np.int32)

In [12]:
pcd_to_las(coord, f"/workspaces/baseline/exp/temporary_export/semantic_KITTI_{frame_nb}.las", gt_segm=segment)